<a href="https://colab.research.google.com/github/leomaurodesenv/big-qa-architecture/blob/main/jupyter/2_Document_Retriever_Experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Document Retriever Experiments

In Question Answering (QA), queries are run over several documents to extract an answer to user questions, consisting of two main steps: (1) Document Retriever — retrieve the most useful documents that may contain the answer to a given question; (2) Document Reader — a machine reader carefully examines the retrieved documents and frame an answer.

In this Jupyter Notebook, we focused in Document Retriever experiments, motivated by the fact that the use of a higher recall algorithm provides a higher end-to-end querying and answering performance.

Blog post: [Automatic Question Answering — Document Retriever (Machine Learning)](https://medium.com/wearesinch/automatic-question-answering-document-retriever-machine-learning-f4f473387739)


---
## Setup

Packages installation and setups.

### Package Installation

Install Haystack and HuggingFace packages.

In [1]:
# Check if you have a GPU running
# The code runs in CPU as well
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [2]:
# %%capture
# Install the Haystack
!pip install pip==22.2.2 --quiet
!pip install farm-haystack[colab]==1.8.0 --quiet
# !pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

# Install Huggingface
!pip install datasets==2.16.1 --quiet
!pip install transformers==4.20.1 --quiet
!pip install sentence-transformers==2.2.2 --quiet
!echo "Silent installation with success!"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.1/330.1 kB 28.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
farm-haystack 1.8.0 requires huggingface-hub<0.8.0,>=0.5.0, but you have huggingface-hub 0.20.3 which is incompatible.
Silent installation with success!


### Logging

We configure how logging messages should be displayed and which log level should be used before importing Haystack.

In [3]:
import logging

# Setup Haystack logging format
logging.basicConfig(format="%(levelname)s - %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

### Run Configuration

Choose the dataset and the Document Retriever algorithm.

In [4]:
import enum

class Dataset(enum.Enum):
    '''Dataset options'''
    SQuAD = 1
    AdvQA = 2
    DuoRC = 3
    QASports = 4

class DocRetriever(enum.Enum):
    '''Document Retriever options'''
    BM25  = 1
    TFIDF = 2
    DPR   = 3

In [5]:
# run configuration
NUM_K         = 1 # 1, 3, 5, 10, 20
DATASET       = Dataset.QASports
DOC_RETRIEVER = DocRetriever.BM25

---
## Document Store

We are going to use Elasticsearch as Document Store; Elasticsearch supports queries using [full-text based](https://www.elastic.co/guide/en/elasticsearch/reference/current/full-text-queries.html), [BM25 retrieval](https://www.elastic.co/elasticon/conf/2016/sf/improved-text-scoring-with-bm25), and [vector space for text embeddings](https://www.elastic.co/guide/en/elasticsearch/reference/7.6/dense-vector.html).

### Starting the Elasticsearch
We manually download and execute the Elasticsearch server.

In [6]:
# In Colab / No Docker environments: Start Elasticsearch from source
!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
!tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
!chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT

es_server = Popen(
    ["elasticsearch-7.9.2/bin/elasticsearch"], stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1)  # as daemon
)
# wait until ES has started
!sleep 30

In [7]:
# Connect to Elasticsearch
from haystack.document_stores import ElasticsearchDocumentStore

document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")

---
## Dataset

Imports and downloads the respective dataset.

### Abstract Dataset

In [8]:
import pandas as pd
from abc import ABCMeta, abstractmethod

class AbstactDataset(metaclass = ABCMeta):
    '''Abstract dataset class'''

    def __init__(self):
        self.raw_dataset = self.download()
        self.df_dataset = self._transform_df()
        print(f"## {self.name} ##")
        print(self.raw_dataset)

    def _transform_df(self):
        '''Transform dataset in a pd.DataFrame'''
        return pd.DataFrame(self.raw_dataset)

    @property
    @abstractmethod
    def name(self):
        '''Dataset name'''
        pass

    @abstractmethod
    def download(self):
        '''Download the dataset'''
        pass

    @abstractmethod
    def get_documents(self):
        '''Get the unique documents to store into the Document Store'''
        pass


    @abstractmethod
    def get_validation(self):
        '''Get the validation set'''
        pass

### SQuaD Dataset

https://huggingface.co/datasets/squad

In [9]:
import mmh3
from datasets import load_dataset
from haystack.schema import Label, Document, Answer
from haystack.schema import EvaluationResult, MultiLabel

class SQuadDataset(AbstactDataset):
    '''SQuaD Dataset'''
    name = "SQuaD Dataset"
    _columns = {
        "title": "title",
        "document": "context",
        "question": "question",
    }
    _metadata = {
        "dataset_id": "id"
    }

    def download(self):
        return load_dataset("squad", split="validation")

    def get_documents(self):
        # Remove duplicated contents
        cc = self._columns
        dataset_name = f"{self.name}"
        df = self.df_dataset
        df = df.drop_duplicates(subset=[cc["title"], cc["document"]], keep="first")

        # Create Haystack Document objects
        list_docs = []
        for _, row in df.iterrows():
            document_id = mmh3.hash128(row[cc["document"]], signed=False)
            doc_metadata = {k: row[v] for k,v in self._metadata.items()}
            doc_metadata["title"] = row[cc["title"]]
            doc_metadata["dataset_name"] = dataset_name
            doc = Document(
                id=document_id,
                content_type="text",
                content=row[cc["document"]],
                meta=doc_metadata
            )
            list_docs.append(doc)
        return list_docs

    def _get_answer(self, data):
        # Get question answer
        return data["answers"]["text"][0]

    def get_validation(self):
        # Get dataset info
        cc = self._columns
        df = self.df_dataset
        _self = self

        # Create Haystack labels
        eval_labels = []
        for _, row in df.iterrows():
            document_id = mmh3.hash128(row[cc["document"]], signed=False)
            doc_label = MultiLabel(labels=[
                Label(
                    query = row[cc["question"]],
                    answer = Answer(
                        answer=_self._get_answer(row),
                        type="extractive",
                    ),
                    document = Document(
                        id=document_id,
                        content_type="text",
                        content=row[cc["document"]],
                    ),
                    is_correct_answer=True,
                    is_correct_document=True,
                    origin="gold-label",
                )
            ])
            eval_labels.append(doc_label)
        return eval_labels

### AdversarialQA Dataset

https://huggingface.co/datasets/adversarial_qa

In [10]:
class AdversarialQADataset(SQuadDataset):
    '''AdversarialQA Dataset'''
    name = "AdversarialQA Dataset"

    def download(self):
        return load_dataset("adversarial_qa", "adversarialQA", split="validation")

### DuoRC Dataset

https://huggingface.co/datasets/duorc

In [11]:
class DuoRCDataset(SQuadDataset):
    '''DuoRC  Dataset'''
    name = "DuoRC Dataset"
    _columns = {
        "title": "title",
        "document": "plot",
        "question": "question",
    }
    _metadata = {
        "dataset_id": "question_id"
    }

    def download(self):
        return load_dataset("duorc", "SelfRC", split="validation")

    def _transform_df(self):
        '''Transform dataset in a pd.DataFrame'''
        df = pd.DataFrame(self.raw_dataset)
        # Get questions with answer
        return df[~df["no_answer"]]

    def _get_answer(self, data):
        # Get question answer
        return data["answers"][0]

### QASports Dataset / Basketball

https://huggingface.co/datasets/PedroCJardim/QASports

In [12]:
class QASportsDataset(SQuadDataset):
    '''QASports  Dataset'''
    name = "QASports Dataset"
    _columns = {
        "title": "context_title",
        "document": "context",
        "question": "question",
    }
    _metadata = {
        "dataset_id": "id_qa"
    }

    def download(self):
        return load_dataset("PedroCJardim/QASports", "basketball", split="validation")

    def _transform_df(self):
        '''Transform dataset in a pd.DataFrame'''
        df = pd.DataFrame(self.raw_dataset)
        # Get questions with answer
        df["answer"] = df["answer"].apply(eval)
        mask = df["answer"].apply(lambda x: True if x["text"] != "" else False)
        return df[mask]

    def _get_answer(self, data):
        # Get question answer
        return data["answer"]["text"]

### Download the dataset

Get the dataset and store its documents into the Document Store.

In [13]:
def dataset_switch(choice):
    '''Get dataset class'''

    if choice == Dataset.SQuAD:
        return SQuadDataset()
    elif choice == Dataset.AdvQA:
        return AdversarialQADataset()
    elif choice == Dataset.DuoRC:
        return DuoRCDataset()
    elif choice == Dataset.QASports:
        return QASportsDataset()
    else:
        return "Invalid dataset"

# Get the dataset
dataset = dataset_switch(DATASET)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

## QASports Dataset ##
Dataset({
    features: ['id_qa', 'context_id', 'context', 'question', 'answer', 'context_title', 'context_categories', 'url'],
    num_rows: 23242
})


In [14]:
# Store documents in the Document Store
docs = dataset.get_documents()
document_store.write_documents(docs)

---
## Document Retriever

In this experiment, we explored the BM25, TF-IDF and Dense Passage Retrieval (DPR).

* https://docs.haystack.deepset.ai/docs/retriever
* https://github.com/facebookresearch/DPR
* https://www.elastic.co/pt/blog/practical-bm25-part-2-the-bm25-algorithm-and-its-variables

### Get the Retriever

In [15]:
from haystack.nodes import BM25Retriever
from haystack.nodes import TfidfRetriever
from haystack.nodes import DensePassageRetriever

def retriever_switch(choice, document_store):
    '''Get Retriever object'''

    if choice == DocRetriever.BM25:
        retriever = BM25Retriever(document_store=document_store)
        return retriever
    elif choice == DocRetriever.TFIDF:
        retriever = TfidfRetriever(document_store=document_store)
        return retriever
    elif choice == DocRetriever.DPR:
        retriever = DensePassageRetriever(
            document_store=document_store,
            query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
            passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
            use_fast_tokenizers=True
        )
        document_store.update_embeddings(retriever)
        return retriever
    else:
        return "Invalid retriever"

# Get the retriever
retriever = retriever_switch(DOC_RETRIEVER, document_store)
retriever

### Build the Pipeline

In [16]:
from haystack.pipelines import DocumentSearchPipeline

pipe = DocumentSearchPipeline(retriever=retriever)

In [17]:
# Testing the pipeline
from haystack.utils import print_documents

# Querying documents
question = "What is your name?"
prediction = pipe.run(query=question, params={"Retriever": {"top_k": 1}})

# Print answer
print_documents(prediction)


Query: What is your name?

{   'content': " As a testament to Cousy's legacy, President John F. Kennedy "
               'wired to Cousy: The game bears the indelible stamp of your '
               'rare skills and competitive daring, and it will serve as a '
               'living reminder of your long and illustrious career so long as '
               'it is played."',
    'name': None}



---
## Evaluation

About the metrics, you can read the [evaluation](https://docs.haystack.deepset.ai/docs/evaluation) web page.

In [18]:
%%time

# For testing purposes, running on the first 100 labels
# For real evaluation, you must remove the [0:100]
eval_labels = dataset.get_validation()[0:10]
eval_result = pipe.eval(labels=eval_labels, params={"Retriever": {"top_k": NUM_K}})

CPU times: user 7.27 s, sys: 196 ms, total: 7.46 s
Wall time: 7.81 s


In [19]:
from pprint import pprint

# Get and print the metrics
metrics = eval_result.calculate_metrics()
pprint(metrics)

{'Retriever': {'map': 0.55,
               'mrr': 0.6,
               'ndcg': 0.5613147192765459,
               'precision': 0.6,
               'recall_multi_hit': 0.55,
               'recall_single_hit': 0.6}}


In [20]:
# Print a detailed report
# pipe.print_eval_report(eval_result)